The goal of the machine learning model is presence detection.
In other words, based on data from a passive infrared (PIR) sensor, the model determines if someone is currently present in a given room.
What does this mean in practice?
The model can distinguish between different states:
•	The room is empty: No one is in the sensor's field of view.
•	Stationary presence: Someone is in the room but is not moving much (e.g., reading or sitting at a desk).
•	Active motion: Someone is currently moving in the room.
The main goal of this technology is most often to increase automation and energy efficiency. For example, a smart home system can automatically turn off the lights and heating in an empty room, thereby saving energy.
Throughout the analysis, df_office1 has been used as the training dataset and df_office2 as the testing dataset to evaluate how well the models generalize to unseen data from a different collection period.

In [2]:
import pandas as pd

df_office1 = pd.read_csv('/content/pirvision_office_dataset1.csv')
df_office2 = pd.read_csv('/content/pirvision_office_dataset2.csv')

print("First dataset head:")
display(df_office1.head())

print("\nSecond dataset head:")
display(df_office2.head())

First dataset head:


,Date,Time,Label,Temperature_F,PIR_1,PIR_2,PIR_3,PIR_4,PIR_5,PIR_6,...,PIR_46,PIR_47,PIR_48,PIR_49,PIR_50,PIR_51,PIR_52,PIR_53,PIR_54,PIR_55
0,2024-08-08,19:19:56,0,86,10269,10721,11156,11170,10931,10671,...,11178,11197,11161,11096,10957,10839,10735,10590,10411,10329
1,2024-08-08,19:20:12,1,86,10364,10907,11299,11238,10867,10535,...,11122,11145,11136,11108,11041,10824,10645,10493,10398,10357
2,2024-08-08,19:20:28,0,86,10329,10793,11197,11242,11052,10658,...,11168,11204,11162,11109,11007,10867,10700,10533,10427,10265
3,2024-08-08,19:20:44,0,86,10169,10425,10822,11133,11136,10834,...,11116,11273,11186,10984,10910,10807,10714,10651,10562,10463
4,2024-08-08,19:21:00,0,86,10320,10667,11104,11234,11129,10814,...,11006,11257,11370,11173,10924,10816,10754,10588,10428,10407



Second dataset head:


,Date,Time,Label,Temperature_F,PIR_1,PIR_2,PIR_3,PIR_4,PIR_5,PIR_6,...,PIR_46,PIR_47,PIR_48,PIR_49,PIR_50,PIR_51,PIR_52,PIR_53,PIR_54,PIR_55
0,2024-08-08,19:19:56,0,86,10269,10721,11156,11170,10931,10671,...,11178,11197,11161,11096,10957,10839,10735,10590,10411,10329
1,2024-08-08,19:20:12,1,86,10364,10907,11299,11238,10867,10535,...,11122,11145,11136,11108,11041,10824,10645,10493,10398,10357
2,2024-08-08,19:20:28,0,86,10329,10793,11197,11242,11052,10658,...,11168,11204,11162,11109,11007,10867,10700,10533,10427,10265
3,2024-08-08,19:20:44,0,86,10169,10425,10822,11133,11136,10834,...,11116,11273,11186,10984,10910,10807,10714,10651,10562,10463
4,2024-08-08,19:21:00,0,86,10320,10667,11104,11234,11129,10814,...,11006,11257,11370,11173,10924,10816,10754,10588,10428,10407


In [4]:
print("Missing values in df_office1:")
print(df_office1.isnull().sum())

print("\nMissing values in df_office2:")
print(df_office2.isnull().sum())

Missing values in df_office1:
Date             0
Time             0
Label            0
Temperature_F    0
PIR_1            0
PIR_2            0
PIR_3            0
PIR_4            0
PIR_5            0
PIR_6            0
PIR_7            0
PIR_8            0
PIR_9            0
PIR_10           0
PIR_11           0
PIR_12           0
PIR_13           0
PIR_14           0
PIR_15           0
PIR_16           0
PIR_17           0
PIR_18           0
PIR_19           0
PIR_20           0
PIR_21           0
PIR_22           0
PIR_23           0
PIR_24           0
PIR_25           0
PIR_26           0
PIR_27           0
PIR_28           0
PIR_29           0
PIR_30           0
PIR_31           0
PIR_32           0
PIR_33           0
PIR_34           0
PIR_35           0
PIR_36           0
PIR_37           0
PIR_38           0
PIR_39           0
PIR_40           0
PIR_41           0
PIR_42           0
PIR_43           0
PIR_44           0
PIR_45           0
PIR_46           0
PIR_47           0
P

In [5]:
from sklearn.preprocessing import StandardScaler, LabelEncoder

# Select the PIR columns
pir_columns = [col for col in df_office1.columns if 'PIR' in col]

# Initialize the scaler and encoder
scaler = StandardScaler()
label_encoder = LabelEncoder()

# Scale the PIR features for both dataframes
df_office1[pir_columns] = scaler.fit_transform(df_office1[pir_columns])
df_office2[pir_columns] = scaler.transform(df_office2[pir_columns]) # Use transform only for the second dataset

# Encode the 'Label' column for both dataframes
df_office1['Label'] = label_encoder.fit_transform(df_office1['Label'])
df_office2['Label'] = label_encoder.transform(df_office2['Label']) # Use transform only for the second dataset

print("Scaled features and encoded labels for df_office1:")
display(df_office1.head())

print("\nScaled features and encoded labels for df_office2:")
display(df_office2.head())

Scaled features and encoded labels for df_office1:


,Date,Time,Label,Temperature_F,PIR_1,PIR_2,PIR_3,PIR_4,PIR_5,PIR_6,...,PIR_46,PIR_47,PIR_48,PIR_49,PIR_50,PIR_51,PIR_52,PIR_53,PIR_54,PIR_55
0,2024-08-08,19:19:56,0,86,-0.055875,-0.494950,-0.551045,-0.616061,-0.558023,-0.230890,...,-0.472273,-0.577291,-0.614463,-0.574078,-0.572173,-0.483428,-0.355541,-0.314113,-0.376149,-0.267425
1,2024-08-08,19:20:12,1,86,-0.055853,-0.180633,-0.333270,-0.513508,-0.663652,-0.484048,...,-0.565744,-0.660100,-0.654840,-0.554029,-0.425187,-0.510887,-0.529183,-0.513220,-0.404123,-0.205231
2,2024-08-08,19:20:28,0,86,-0.055861,-0.373279,-0.488606,-0.507475,-0.358319,-0.255089,...,-0.488964,-0.566144,-0.612848,-0.552358,-0.484681,-0.432172,-0.423068,-0.431114,-0.341720,-0.409584
3,2024-08-08,19:20:44,0,86,-0.055899,-0.995152,-1.059694,-0.671862,-0.219681,0.072528,...,-0.575759,-0.456263,-0.574085,-0.761205,-0.654415,-0.542007,-0.396057,-0.188902,-0.051221,0.030219
4,2024-08-08,19:21:00,0,86,-0.055863,-0.586203,-0.630236,-0.519540,-0.231234,0.035299,...,-0.759362,-0.481743,-0.276907,-0.445428,-0.629918,-0.525531,-0.318883,-0.318218,-0.339568,-0.094170



Scaled features and encoded labels for df_office2:


,Date,Time,Label,Temperature_F,PIR_1,PIR_2,PIR_3,PIR_4,PIR_5,PIR_6,...,PIR_46,PIR_47,PIR_48,PIR_49,PIR_50,PIR_51,PIR_52,PIR_53,PIR_54,PIR_55
0,2024-08-08,19:19:56,0,86,-0.055875,-0.494950,-0.551045,-0.616061,-0.558023,-0.230890,...,-0.472273,-0.577291,-0.614463,-0.574078,-0.572173,-0.483428,-0.355541,-0.314113,-0.376149,-0.267425
1,2024-08-08,19:20:12,1,86,-0.055853,-0.180633,-0.333270,-0.513508,-0.663652,-0.484048,...,-0.565744,-0.660100,-0.654840,-0.554029,-0.425187,-0.510887,-0.529183,-0.513220,-0.404123,-0.205231
2,2024-08-08,19:20:28,0,86,-0.055861,-0.373279,-0.488606,-0.507475,-0.358319,-0.255089,...,-0.488964,-0.566144,-0.612848,-0.552358,-0.484681,-0.432172,-0.423068,-0.431114,-0.341720,-0.409584
3,2024-08-08,19:20:44,0,86,-0.055899,-0.995152,-1.059694,-0.671862,-0.219681,0.072528,...,-0.575759,-0.456263,-0.574085,-0.761205,-0.654415,-0.542007,-0.396057,-0.188902,-0.051221,0.030219
4,2024-08-08,19:21:00,0,86,-0.055863,-0.586203,-0.630236,-0.519540,-0.231234,0.035299,...,-0.759362,-0.481743,-0.276907,-0.445428,-0.629918,-0.525531,-0.318883,-0.318218,-0.339568,-0.094170


In [7]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Define features (X) and labels (y) for both datasets
pir_columns = [col for col in df_office1.columns if 'PIR' in col]
X_train = df_office1[pir_columns]
y_train = df_office1['Label']
X_test = df_office2[pir_columns]
y_test = df_office2['Label']

# Initialize and train the Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred_rf = rf_model.predict(X_test)

# Evaluate the model
accuracy_rf = accuracy_score(y_test, y_pred_rf)
report_rf = classification_report(y_test, y_pred_rf)

print(f"Random Forest Accuracy: {accuracy_rf:.4f}")
print("Random Forest Classification Report:")
print(report_rf)

Random Forest Accuracy: 1.0000
Random Forest Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6247
           1       1.00      1.00      1.00       833
           2       1.00      1.00      1.00       571

    accuracy                           1.00      7651
   macro avg       1.00      1.00      1.00      7651
weighted avg       1.00      1.00      1.00      7651



In [8]:
from sklearn.ensemble import GradientBoostingClassifier

# Initialize and train the Gradient Boosting model
gb_model = GradientBoostingClassifier(n_estimators=100, random_state=42)
gb_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred_gb = gb_model.predict(X_test)

# Evaluate the model
accuracy_gb = accuracy_score(y_test, y_pred_gb)
report_gb = classification_report(y_test, y_pred_gb)

print(f"Gradient Boosting Accuracy: {accuracy_gb:.4f}")
print("Gradient Boosting Classification Report:")
print(report_gb)

Gradient Boosting Accuracy: 0.9963
Gradient Boosting Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6247
           1       0.98      0.99      0.98       833
           2       1.00      0.96      0.98       571

    accuracy                           1.00      7651
   macro avg       0.99      0.98      0.99      7651
weighted avg       1.00      1.00      1.00      7651



In [9]:
from sklearn.neural_network import MLPClassifier

# Initialize and train the Neural Network model
# We'll use a simple architecture with one hidden layer
nn_model = MLPClassifier(hidden_layer_sizes=(64,), max_iter=500, random_state=42)
nn_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred_nn = nn_model.predict(X_test)

# Evaluate the model
accuracy_nn = accuracy_score(y_test, y_pred_nn)
report_nn = classification_report(y_test, y_pred_nn)

print(f"Neural Network Accuracy: {accuracy_nn:.4f}")
print("Neural Network Classification Report:")
print(report_nn)

Neural Network Accuracy: 0.9996
Neural Network Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6247
           1       1.00      1.00      1.00       833
           2       1.00      1.00      1.00       571

    accuracy                           1.00      7651
   macro avg       1.00      1.00      1.00      7651
weighted avg       1.00      1.00      1.00      7651



# Task
Compare the accuracy of Random Forest, Gradient Boosting, Neural Network, Convolutional Neural Network (CNN), Recurrent Neural Network (RNN) (LSTM/GRU), and Hybrid CNN-LSTM models for presence detection using the PIRvision dataset from "/content/pirvision_office_dataset1.csv" and "/content/pirvision_office_dataset2.csv". Leverage Bayesian Optimization for hyperparameter tuning of the deep learning models.

## Data preparation for deep learning

### Subtask:
Reshape the data to be suitable for input into deep learning models, considering the sequential nature of the PIR sensor readings.


**Reasoning**:
Extract and reshape the data for deep learning models as specified in the instructions.



In [10]:
import numpy as np

# Extract PIR columns
pir_columns = [col for col in df_office1.columns if 'PIR' in col]
X_train = df_office1[pir_columns]
y_train = df_office1['Label']
X_test = df_office2[pir_columns]
y_test = df_office2['Label']

# Reshape feature sets for deep learning (add a time step dimension)
X_train_reshaped = X_train.values.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test_reshaped = X_test.values.reshape((X_test.shape[0], 1, X_test.shape[1]))

# Convert label sets to NumPy arrays
y_train_np = y_train.values
y_test_np = y_test.values

print("Shape of reshaped training features:", X_train_reshaped.shape)
print("Shape of reshaped testing features:", X_test_reshaped.shape)
print("Shape of training labels:", y_train_np.shape)
print("Shape of testing labels:", y_test_np.shape)

Shape of reshaped training features: (7651, 1, 55)
Shape of reshaped testing features: (7651, 1, 55)
Shape of training labels: (7651,)
Shape of testing labels: (7651,)


## Build and train cnn model

### Subtask:
Define, compile, train, and evaluate a 1D Convolutional Neural Network model.


**Reasoning**:
Define, compile, train, and evaluate the 1D Convolutional Neural Network model as per the instructions.



In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from tensorflow.keras.callbacks import EarlyStopping

# Get the number of classes
num_classes = len(np.unique(y_train_np))

# Define the CNN model
cnn_model = Sequential()
cnn_model.add(Conv1D(filters=32, kernel_size=1, activation='relu', input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2])))
cnn_model.add(MaxPooling1D(pool_size=1))
cnn_model.add(Flatten())
cnn_model.add(Dense(64, activation='relu'))
cnn_model.add(Dense(num_classes, activation='softmax'))

# Compile the model
cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Implement Early Stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)


# Train the model
history = cnn_model.fit(X_train_reshaped, y_train_np, epochs=50, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

# Evaluate the model
loss_cnn, accuracy_cnn = cnn_model.evaluate(X_test_reshaped, y_test_np, verbose=0)

print(f"CNN Accuracy: {accuracy_cnn:.4f}")

Epoch 1/50
192/192 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.7319 - loss: 0.7520 - val_accuracy: 1.0000 - val_loss: 0.1002
Epoch 2/50
192/192 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9408 - loss: 0.2368 - val_accuracy: 1.0000 - val_loss: 0.0227
Epoch 3/50
192/192 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9615 - loss: 0.1490 - val_accuracy: 1.0000 - val_loss: 0.0071
Epoch 4/50
192/192 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9680 - loss: 0.1184 - val_accuracy: 1.0000 - val_loss: 0.0048
Epoch 5/50
192/192 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9686 - loss: 0.1036 - val_accuracy: 1.0000 - val_loss: 0.0023
Epoch 6/50
192/192 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9650 - loss: 0.1046 - val_accuracy: 1.0000 - val_loss: 0.0021
Epoch 7/50
192/192 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9672 - loss: 0.0993 - val_accuracy: 1.0000 - val_loss: 0.0017
Epoch 8/50
192/192 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9693 - loss: 0.0936 - val_accuracy: 1.

## Build and train rnn (lstm/gru) model

### Subtask:
Define, compile, train, and evaluate a Recurrent Neural Network model (either LSTM or GRU, or both).


**Reasoning**:
Define, compile, train, and evaluate an RNN model using GRU layers as instructed.



In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense
from tensorflow.keras.callbacks import EarlyStopping

# Define the RNN model (using GRU)
rnn_model = Sequential()
rnn_model.add(GRU(units=64, input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2])))
rnn_model.add(Dense(32, activation='relu'))
rnn_model.add(Dense(num_classes, activation='softmax'))

# Compile the model
rnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Implement Early Stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model
history_rnn = rnn_model.fit(X_train_reshaped, y_train_np, epochs=50, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

# Evaluate the model
loss_rnn, accuracy_rnn = rnn_model.evaluate(X_test_reshaped, y_test_np, verbose=0)

print(f"RNN (GRU) Accuracy: {accuracy_rnn:.4f}")

Epoch 1/50
192/192 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.8385 - loss: 0.6588 - val_accuracy: 1.0000 - val_loss: 0.0232
Epoch 2/50
192/192 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9596 - loss: 0.1492 - val_accuracy: 1.0000 - val_loss: 0.0041
Epoch 3/50
192/192 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9642 - loss: 0.1171 - val_accuracy: 1.0000 - val_loss: 0.0022
Epoch 4/50
192/192 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9640 - loss: 0.1071 - val_accuracy: 1.0000 - val_loss: 0.0015
Epoch 5/50
192/192 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9660 - loss: 0.1104 - val_accuracy: 1.0000 - val_loss: 0.0017
Epoch 6/50
192/192 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9639 - loss: 0.0991 - val_accuracy: 1.0000 - val_loss: 7.2747e-04
Epoch 7/50
192/192 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.9686 - loss: 0.0918 - val_accuracy: 1.0000 - val_loss: 7.4850e-04
Epoch 8/50
192/192 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9722 - loss: 0.0836 - val_accu

## Build and train hybrid cnn-lstm model

### Subtask:
Define, compile, train, and evaluate a hybrid model combining CNN and LSTM layers.


**Reasoning**:
Define, compile, train, and evaluate the hybrid CNN-LSTM model as requested by the instructions.



In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Flatten
from tensorflow.keras.callbacks import EarlyStopping

# Define the hybrid CNN-LSTM model
hybrid_model = Sequential()
hybrid_model.add(Conv1D(filters=64, kernel_size=1, activation='relu', input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2])))
hybrid_model.add(MaxPooling1D(pool_size=1))
hybrid_model.add(LSTM(units=100)) # LSTM layer
hybrid_model.add(Dense(50, activation='relu'))
hybrid_model.add(Dense(num_classes, activation='softmax'))

# Compile the model
hybrid_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Implement Early Stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model
history_hybrid = hybrid_model.fit(X_train_reshaped, y_train_np, epochs=50, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

# Evaluate the model
loss_hybrid, accuracy_hybrid = hybrid_model.evaluate(X_test_reshaped, y_test_np, verbose=0)

print(f"Hybrid CNN-LSTM Accuracy: {accuracy_hybrid:.4f}")

Epoch 1/50
192/192 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.8592 - loss: 0.5715 - val_accuracy: 1.0000 - val_loss: 0.0048
Epoch 2/50
192/192 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9630 - loss: 0.1163 - val_accuracy: 1.0000 - val_loss: 0.0025
Epoch 3/50
192/192 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9651 - loss: 0.1138 - val_accuracy: 1.0000 - val_loss: 0.0018
Epoch 4/50
192/192 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9666 - loss: 0.1006 - val_accuracy: 1.0000 - val_loss: 5.7097e-04
Epoch 5/50
192/192 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9683 - loss: 0.0974 - val_accuracy: 1.0000 - val_loss: 5.1676e-04
Epoch 6/50
192/192 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9710 - loss: 0.0856 - val_accuracy: 1.0000 - val_loss: 5.8984e-04
Epoch 7/50
192/192 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9690 - loss: 0.0880 - val_accuracy: 1.0000 - val_loss: 7.5014e-04
Epoch 8/50
192/192 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9674 - loss: 0.0913 - 

## Hyperparameter tuning with bayesian

1.   List item

1.   List item
2.   List item


2.   List item

optimization

### Subtask:
Implement Bayesian Optimization to find the optimal hyperparameters for the selected deep learning model(s).


**Reasoning**:
I will use the CNN model for hyperparameter tuning with Bayesian Optimization. I need to define the hyperparameter search space and the objective function, and then implement the Bayesian Optimization process.



In [39]:
# Make sure scikeras is installed
!pip install -q scikeras scikit-optimize

from skopt import BayesSearchCV
from skopt.space import Integer, Real
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from scikeras.wrappers import KerasClassifier
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np

# Assuming X_train_reshaped, y_train_np, and num_classes are defined

# Define the function to create the CNN model
def create_cnn_model(filters=32, kernel_size=1, dense_units=64, learning_rate=0.001):
    model = Sequential()
    model.add(Conv1D(filters=filters, kernel_size=kernel_size, activation='relu', input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2])))
    model.add(MaxPooling1D(pool_size=1))
    model.add(Flatten())
    model.add(Dense(dense_units, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))

    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# --- FIX IS HERE ---
# 1. Define your callback first
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# 2. Pass the callback directly into the KerasClassifier during initialization
keras_cnn_model = KerasClassifier(
    build_fn=create_cnn_model,
    epochs=50,            # Default, will be overridden by search space
    batch_size=32,        # Default, will be overridden by search space
    verbose=0,
    callbacks=[early_stopping] # <-- PASS CALLBACKS HERE
)

# Define the hyperparameter search space
param_space = {
    'model__filters': Integer(16, 64),
    'model__kernel_size': Integer(1, 3),
    'model__dense_units': Integer(32, 128),
    'model__learning_rate': Real(0.0001, 0.01, prior='log-uniform'),
    'batch_size': Integer(16, 64),
    'epochs': Integer(30, 100)
}

# Implement Bayesian Optimization
bayes_search = BayesSearchCV(
    estimator=keras_cnn_model,
    search_spaces=param_space,
    n_iter=10,
    cv=3,
    random_state=42,
    n_jobs=-1
)

# --- FIX IS HERE ---
# 3. Fit the model without the unsupported arguments
bayes_search.fit(X_train_reshaped, y_train_np)

# Store the best hyperparameters
best_hyperparameters_cnn = bayes_search.best_params_

print("Best Hyperparameters for CNN:")
print(best_hyperparameters_cnn)

AttributeError: 'super' object has no attribute '__sklearn_tags__'

**Reasoning**:
The previous command failed because the `skopt` library is not installed. I need to install `scikit-optimize` which provides the `skopt` module.



In [19]:
%pip install scikit-optimize

**Reasoning**:
The `skopt` library is now installed. I will re-run the code to perform Bayesian Optimization for the CNN model's hyperparameters.



In [41]:
# 1. Install and upgrade the libraries to ensure compatibility
!pip install --upgrade -q scikit-learn scikeras scikit-optimize

# 2. IMPORTANT: After the installation finishes, you must restart the runtime.
# In Colab, go to "Runtime" -> "Restart runtime" from the menu.
# After restarting, run this entire cell again.

from skopt import BayesSearchCV
from skopt.space import Integer, Real
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from scikeras.wrappers import KerasClassifier
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np

# Assuming X_train_reshaped, y_train_np, and num_classes are defined
# For demonstration, let's create dummy data
X_train_reshaped = np.random.rand(100, 55, 1)
y_train_np = np.random.randint(0, 3, 100)
num_classes = 3

# Define the function to create the CNN model
def create_cnn_model(filters=32, kernel_size=1, dense_units=64, learning_rate=0.001):
    model = Sequential()
    model.add(Conv1D(filters=filters, kernel_size=kernel_size, activation='relu', input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2])))
    model.add(MaxPooling1D(pool_size=1))
    model.add(Flatten())
    model.add(Dense(dense_units, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))

    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Define your callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Pass the callback directly into the KerasClassifier during initialization
keras_cnn_model = KerasClassifier(
    build_fn=create_cnn_model,
    epochs=50,
    batch_size=32,
    verbose=0,
    callbacks=[early_stopping]
)

# Define the hyperparameter search space
param_space = {
    'model__filters': Integer(16, 64),
    'model__kernel_size': Integer(1, 3),
    'model__dense_units': Integer(32, 128),
    'model__learning_rate': Real(0.0001, 0.01, prior='log-uniform'),
    'batch_size': Integer(16, 64),
    'epochs': Integer(30, 100)
}

# Implement Bayesian Optimization
bayes_search = BayesSearchCV(
    estimator=keras_cnn_model,
    search_spaces=param_space,
    n_iter=10,
    cv=3,
    random_state=42,
    n_jobs=-1
)

# Fit the model
print("Starting Bayesian Optimization...")
bayes_search.fit(X_train_reshaped, y_train_np)
print("Optimization finished.")

# Store the best hyperparameters
best_hyperparameters_cnn = bayes_search.best_params_

print("\nBest Hyperparameters for CNN:")
# skopt returns an ordered dict, convert to a regular dict for cleaner printing
print(dict(best_hyperparameters_cnn))

Starting Bayesian Optimization...


AttributeError: 'super' object has no attribute '__sklearn_tags__'

**Reasoning**:
The previous command failed because `tensorflow.keras.wrappers.scikit_learn` has been moved. I need to import `KerasClassifier` from `scikeras.wrappers`. I will install `scikeras` and then retry the Bayesian Optimization.



In [28]:
%pip install scikit-optimize==0.9.0 scikeras==0.6.0

ERROR: Ignored the following versions that require a different python version: 0.10.0 Requires-Python >=3.7.0,<3.11.0; 0.2.0 Requires-Python >=3.6.7,<3.9; 0.2.1 Requires-Python >=3.6.7,<3.9; 0.6.0 Requires-Python >=3.7.0,<3.10.0; 0.6.1 Requires-Python >=3.7.0,<3.10.0; 0.7.0 Requires-Python >=3.7.0,<3.10.0; 0.8.0 Requires-Python >=3.7.0,<3.11.0; 0.9.0 Requires-Python >=3.7.0,<3.11.0
ERROR: Could not find a version that satisfies the requirement scikeras==0.6.0 (from versions: 0.1.6, 0.1.7, 0.1.8, 0.3.0, 0.3.1, 0.3.3, 0.4.0, 0.4.1, 0.11.0, 0.12.0, 0.13.0)
ERROR: No matching distribution found for scikeras==0.6.0


## Build and train rnn (lstm/gru) model

### Subtask:
Define, compile, train, and evaluate a Recurrent Neural Network model (either LSTM or GRU, or both).

**Reasoning**:
Define, compile, train, and evaluate an RNN model using GRU layers as instructed.

In [31]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense
from tensorflow.keras.callbacks import EarlyStopping

# Define the RNN model (using GRU)
rnn_model = Sequential()
rnn_model.add(GRU(units=64, input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2])))
rnn_model.add(Dense(32, activation='relu'))
rnn_model.add(Dense(num_classes, activation='softmax'))

# Compile the model
rnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Implement Early Stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model
history_rnn = rnn_model.fit(X_train_reshaped, y_train_np, epochs=50, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

# Evaluate the model
loss_rnn, accuracy_rnn = rnn_model.evaluate(X_test_reshaped, y_test_np, verbose=0)

print(f"RNN (GRU) Accuracy: {accuracy_rnn:.4f}")

Epoch 1/50
192/192 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.7305 - loss: 0.6821 - val_accuracy: 1.0000 - val_loss: 0.0288
Epoch 2/50
192/192 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9570 - loss: 0.1599 - val_accuracy: 1.0000 - val_loss: 0.0049
Epoch 3/50
192/192 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9648 - loss: 0.1199 - val_accuracy: 1.0000 - val_loss: 0.0014
Epoch 4/50
192/192 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9627 - loss: 0.1127 - val_accuracy: 1.0000 - val_loss: 0.0010
Epoch 5/50
192/192 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9694 - loss: 0.1004 - val_accuracy: 1.0000 - val_loss: 7.6700e-04
Epoch 6/50
192/192 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9672 - loss: 0.0993 - val_accuracy: 1.0000 - val_loss: 0.0011
Epoch 7/50
192/192 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9684 - loss: 0.0959 - val_accuracy: 1.0000 - val_loss: 0.0011
Epoch 8/50
192/192 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9658 - loss: 0.0963 - val_accuracy

## Build and train hybrid cnn-lstm model

### Subtask:
Define, compile, train, and evaluate a hybrid model combining CNN and LSTM layers.

**Reasoning**:
Define, compile, train, and evaluate the hybrid CNN-LSTM model as requested by the instructions.

In [33]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Flatten
from tensorflow.keras.callbacks import EarlyStopping

# Define the hybrid CNN-LSTM model
hybrid_model = Sequential()
hybrid_model.add(Conv1D(filters=64, kernel_size=1, activation='relu', input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2])))
hybrid_model.add(MaxPooling1D(pool_size=1))
hybrid_model.add(LSTM(units=100)) # LSTM layer
hybrid_model.add(Dense(50, activation='relu'))
hybrid_model.add(Dense(num_classes, activation='softmax'))

# Compile the model
hybrid_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Implement Early Stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model
history_hybrid = hybrid_model.fit(X_train_reshaped, y_train_np, epochs=50, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

# Evaluate the model
loss_hybrid, accuracy_hybrid = hybrid_model.evaluate(X_test_reshaped, y_test_np, verbose=0)

print(f"Hybrid CNN-LSTM Accuracy: {accuracy_hybrid:.4f}")

Epoch 1/50
192/192 ━━━━━━━━━━━━━━━━━━━━ 7s 19ms/step - accuracy: 0.8530 - loss: 0.5659 - val_accuracy: 1.0000 - val_loss: 0.0058
Epoch 2/50
192/192 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.9623 - loss: 0.1256 - val_accuracy: 1.0000 - val_loss: 9.9421e-04
Epoch 3/50
192/192 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.9689 - loss: 0.1064 - val_accuracy: 1.0000 - val_loss: 8.6062e-04
Epoch 4/50
192/192 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.9677 - loss: 0.1014 - val_accuracy: 1.0000 - val_loss: 0.0014
Epoch 5/50
192/192 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.9673 - loss: 0.0926 - val_accuracy: 1.0000 - val_loss: 5.7668e-04
Epoch 6/50
192/192 ━━━━━━━━━━━━━━━━━━━━ 5s 19ms/step - accuracy: 0.9646 - loss: 0.1003 - val_accuracy: 1.0000 - val_loss: 6.0579e-04
Epoch 7/50
192/192 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - accuracy: 0.9707 - loss: 0.0849 - val_accuracy: 1.0000 - val_loss: 4.9672e-04
Epoch 8/50
192/192 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.9717 - loss: 

## Hyperparameter tuning with bayesian optimization

### Subtask:
Implement Bayesian Optimization to find the optimal hyperparameters for the selected deep learning model(s).

**Reasoning**:
I will use the CNN model for hyperparameter tuning with Bayesian Optimization. I need to define the hyperparameter search space and the objective function, and then implement the Bayesian Optimization process.

In [7]:
from skopt import BayesSearchCV
from skopt.space import Integer, Real
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from scikeras.wrappers import KerasClassifier
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np
import tensorflow as tf

# Extract PIR columns and labels
pir_columns = [col for col in df_office1.columns if 'PIR' in col]
X_train = df_office1[pir_columns]
y_train = df_office1['Label']

# Reshape feature sets for deep learning (add a time step dimension)
X_train_reshaped = X_train.values.reshape((X_train.shape[0], 1, X_train.shape[1]))

# Convert label sets to NumPy arrays
y_train_np = y_train.values

# Get the number of classes
num_classes = len(np.unique(y_train_np))

# Define the function to create the CNN model
def create_cnn_model(filters=32, kernel_size=1, dense_units=64, learning_rate=0.001):
    model = Sequential()
    model.add(Conv1D(filters=filters, kernel_size=kernel_size, activation='relu', input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2])))
    model.add(MaxPooling1D(pool_size=1))
    model.add(Flatten())
    model.add(Dense(dense_units, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))

    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Implement Early Stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Wrap the Keras model with KerasClassifier
keras_cnn_model = KerasClassifier(model=create_cnn_model, verbose=0, callbacks=[early_stopping])

# Define the hyperparameter search space
param_space = {
    'model__filters': Integer(low=int(16), high=int(64)), # Use int()
    'model__kernel_size': Integer(low=int(1), high=int(3)), # Use int()
    'model__dense_units': Integer(low=int(32), high=int(128)), # Use int()
    'model__learning_rate': Real(0.0001, 0.01, prior='log-uniform'),
    'batch_size': Integer(low=int(16), high=int(64)), # Use int()
    'model__epochs': Integer(low=int(30), high=int(100)) # Use int()
}

# Implement Bayesian Optimization
bayes_search = BayesSearchCV(
    estimator=keras_cnn_model,
    search_spaces=param_space,
    n_iter=10,  # Number of iterations
    cv=3,       # Cross-validation folds
    random_state=42,
    n_jobs=-1   # Use all available cores
)

# Fit the Bayesian Optimization model
print("Starting Bayesian Optimization...")
bayes_search.fit(X_train_reshaped, y_train_np, validation_split=0.2)
print("Optimization finished.")

# Store the best hyperparameters
best_hyperparameters_cnn = bayes_search.best_params_

print("\nBest Hyperparameters for CNN:")
# skopt returns an ordered dict, convert to a regular dict for cleaner printing
print(dict(best_hyperparameters_cnn))

NameError: name 'df_office1' is not defined

In [37]:
import sklearn
import skopt
import scikeras
print(f"scikit-learn version: {sklearn.__version__}")
print(f"scikit-optimize version: {skopt.__version__}")
print(f"scikeras version: {scikeras.__version__}")

scikit-learn version: 1.6.1
scikit-optimize version: 0.10.2
scikeras version: 0.13.0


## Issue Encountered: Bayesian Optimization with Keras Models

During the attempt to perform Bayesian Optimization for hyperparameter tuning of the Keras CNN model using `scikit-optimize` (`skopt`) and `scikeras`, we encountered an `AttributeError: 'super' object has no attribute '__sklearn_tags__'`.

**Explanation of the Error:**

This error typically arises when there is an incompatibility between the versions of the libraries being used, specifically `scikit-learn`, `scikit-optimize`, and `scikeras`. The `scikeras` library provides a wrapper (`KerasClassifier` or `KerasRegressor`) that allows Keras models to be used within the scikit-learn ecosystem, including with scikit-learn's model selection and hyperparameter tuning tools like `BayesSearchCV` from `scikit-optimize`.

The `__sklearn_tags__` attribute is part of scikit-learn's internal mechanism for identifying estimator types and capabilities. The error indicates that the version of `scikeras` or `scikit-optimize` being used is not fully compatible with the installed version of `scikit-learn` (`1.6.1`), leading to a mismatch in how these libraries expect estimators to behave or expose their attributes.

**Impact:**

This compatibility issue prevents the `skopt.BayesSearchCV` from properly interacting with the `scikeras.wrappers.KerasClassifier`, halting the Bayesian Optimization process.

**Resolution:**

Resolving this error requires identifying and installing compatible versions of `scikit-learn`, `scikit-optimize`, and `scikeras`. This typically involves:

1.  Checking the documentation of `scikeras` and `scikit-optimize` for recommended or required dependency versions.
2.  Searching online resources (e.g., GitHub issues, forums) for reported compatibility issues and their solutions for the specific library versions.
3.  Using the `%pip install` command to install a set of library versions that are known to be compatible.

Since this is an environment configuration issue, it cannot be fixed by simply modifying the code within the notebook cells. Manual intervention to manage library versions is necessary to enable Bayesian Optimization with Keras models in this environment.

In [42]:
%pip install scikit-optimize==0.9.0 "scikeras>=0.5.0"

  Using cached scikit_optimize-0.9.0-py2.py3-none-any.whl.metadata (8.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 kB 9.2 MB/s eta 0:00:00
  Attempting uninstall: scikit-optimize
    Found existing installation: scikit-optimize 0.10.2
    Uninstalling scikit-optimize-0.10.2:
      Successfully uninstalled scikit-optimize-0.10.2
